In [4]:
import pandas as pd
import numpy as np

In [2]:
demands = pd.read_csv("data/processed/demands.csv")

In [3]:
demands

,day,fire_id,demand,outside_gacc
0,1,1,1,False
1,19,2,1,False
2,20,3,2,False
3,21,3,2,False
4,22,3,2,False
...,...,...,...,...
418,186,102,1,False
419,186,103,2,False
420,187,104,1,False
421,188,104,1,False


In [76]:
G = 3
J = 10
T = 14

# 0 == rest
crew_inital_locations = [-1, -1, -1, 0, 0, 0, 1, 1, 2, 3]
latest_rest = [14, 14, 7, 2, 3, 3, 4, 5, 6, 10]
crew_current_rest_periods = [2, 4, 1, 0, 0, 0, 0, 0, 0, 0]

rest_length = 2

starting_perims = [10, 4, 4]
L = 2 * np.ones((G, T))

R = [[5, 5, 4, 3, 1, 1, 0.5, 0.3, 0.2, 0.2, 1],
     [8, 8, 8, 4, 1, 1, 1, 0.5, 0.2, 0.2, 0.1], 
     [2, 2, 2, 2, 2, 2, 1.5, 1.5, 1.5, 1, 0.8]]
np.product(R, axis=1)

array([  1.8  ,   4.096, 172.8  ])

In [77]:
base_fire_dists = np.loadtxt("data/processed/base_fire_distances.csv", delimiter=',')
base_fire_dists = base_fire_dists[:10, :4]
fire_dists = np.loadtxt("data/processed/fire_distances.csv", delimiter=',')

tau = 1 + (fire_dists > 100) + (fire_dists > 500)
tau_base = np.ones(base_fire_dists.shape).astype(int)

tau[:G, :G]

array([[1, 2, 2],
       [2, 1, 2],
       [2, 2, 1]])

In [78]:
fire_to_fire_no_rest = [(crew, fire_from, fire_to, period_from, period_from + tau[fire_from, fire_to], 0) 
                           for crew in range(J)
                           for fire_from in range(G)
                           for fire_to in range(G)
                           for period_from in range(T)
                           if period_from <= latest_rest[crew]]

fire_to_fire_rest = [(crew, fire_from, fire_to, period_from, period_from + tau[fire_from, fire_to], 1) 
                        for crew in range(J)
                        for fire_from in range(G)
                        for fire_to in range(G)
                        for period_from in range(T)
                    ]

fire_to_fire_from_start = [(crew, crew_inital_locations[crew], fire_to, -1, 
                            -1 + tau[crew_inital_locations[crew], fire_to], 0) 
                                for crew in range(J)
                                for fire_to in range(G)
                           if crew_inital_locations[crew] != -1
                          ]

In [79]:
fire_to_fire = fire_to_fire_no_rest + fire_to_fire_rest + fire_to_fire_from_start

In [72]:
fire_to_rest_no_rest = [(crew, fire_from, period_from, period_from + tau_base[crew, fire_from], 0) 
                           for crew in range(J)
                           for fire_from in range(G)
                           for fire_to in range(G)
                           for period_from in range(T)
                           if period_from <= latest_rest[crew]
                       ]

fire_to_rest_rest = [(crew, fire_from, period_from, period_from + tau_base[crew, fire_from], 1) 
                        for crew in range(J)
                        for fire_from in range(G)
                        for fire_to in range(G)
                        for period_from in range(T)
                    ]
len(fire_to_rest_no_rest)

684

In [73]:
sum(crew_days_until_rest)

68

In [64]:
len(fire_to_fire)

1965

In [ ]:
starting_perims = 